In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from tqdm import tqdm
from collections import Counter

# ==============================
# CONFIG
# ==============================
DATA_DIR = r"E:\ASL_Citizen\NEW\Top_Classes_Landmarks_Preprocessed_method2"
TARGET_FRAMES = 157
FEATURE_DIM = 438

D_MODEL = 128
N_HEADS = 4
N_LAYERS = 2

BATCH_SIZE = 32
LR = 3e-4
EPOCHS = 60
PATIENCE = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# ==============================
# LOAD FILES + LABELS (FIXED)
# ==============================
files, labels = [], []

for f in os.listdir(DATA_DIR):
    if f.endswith(".npy") and "_mask" not in f:
        files.append(os.path.join(DATA_DIR, f))
        label = f.rsplit("_", 1)[0]  # ✅ FULL GLOSS NAME
        labels.append(label)

print("Total samples:", len(files))

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(labels)
num_classes = len(le.classes_)
print("Num classes:", num_classes)

# ==============================
# DATASET
# ==============================
class LandmarkDataset(Dataset):
    def __init__(self, files, labels, augment=False):
        self.files = files
        self.labels = labels
        self.augment = augment

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x = np.load(self.files[idx]).astype(np.float32)  # (T, F)

        if self.augment:
            noise = np.random.normal(0, 0.002, x.shape).astype(np.float32)
            x = x + noise

        x = torch.tensor(x)               # (T, F)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# ==============================
# SPLIT
# ==============================
X_train, X_tmp, y_train, y_tmp = train_test_split(
    files, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

train_ds = LandmarkDataset(X_train, y_train, augment=True)
val_ds   = LandmarkDataset(X_val, y_val)
test_ds  = LandmarkDataset(X_test, y_test)

# ==============================
# CLASS IMBALANCE HANDLING
# ==============================
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)

sample_weights = np.array([class_weights[y] for y in y_train])
sampler = WeightedRandomSampler(sample_weights, len(sample_weights))

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# ==============================
# MODEL
# ==============================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=300):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class SignTransformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.input_proj = nn.Linear(FEATURE_DIM, D_MODEL)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, D_MODEL))
        self.pos_enc = PositionalEncoding(D_MODEL)

        enc_layer = nn.TransformerEncoderLayer(
            d_model=D_MODEL,
            nhead=N_HEADS,
            dim_feedforward=D_MODEL * 4,
            dropout=0.1,
            batch_first=True
        )

        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=N_LAYERS)

        self.classifier = nn.Sequential(
            nn.LayerNorm(D_MODEL),
            nn.Linear(D_MODEL, num_classes)
        )

    def forward(self, x):
        # x: (B, T, F)
        x = self.input_proj(x)

        B = x.size(0)
        cls = self.cls_token.expand(B, -1, -1)
        x = torch.cat([cls, x], dim=1)

        x = self.pos_enc(x)
        x = self.encoder(x)

        cls_out = x[:, 0]
        return self.classifier(cls_out)

model = SignTransformer().to(DEVICE)

# ==============================
# TRAINING SETUP
# ==============================
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss(weight=class_weights)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# ==============================
# TRAIN LOOP
# ==============================
best_val = 0
pat = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        x, y = x.to(DEVICE), y.to(DEVICE)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    scheduler.step()
    train_acc = correct / total

    # Validation
    model.eval()
    v_correct, v_total = 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            out = model(x)
            v_correct += (out.argmax(1) == y).sum().item()
            v_total += y.size(0)

    val_acc = v_correct / v_total
    print(f"Loss {total_loss:.3f} | Train {train_acc:.4f} | Val {val_acc:.4f}")

    if val_acc > best_val:
        best_val = val_acc
        torch.save(model.state_dict(), "best_sign_transformer.pth")
        pat = 0
        print("  ✅ Model Saved")
    else:
        pat += 1
        if pat >= PATIENCE:
            print("⛔ Early stopping")
            break

print("Best Val Accuracy:", best_val)


Using device: cpu
Total samples: 5568
Num classes: 146


Epoch 1/60: 100%|████████████████████████████████████████████████████████████████████| 140/140 [02:49<00:00,  1.21s/it]


Loss 688.974 | Train 0.0144 | Val 0.0323
  ✅ Model Saved


Epoch 2/60: 100%|████████████████████████████████████████████████████████████████████| 140/140 [01:39<00:00,  1.41it/s]


Loss 577.833 | Train 0.0676 | Val 0.0646
  ✅ Model Saved


...

Loss 11.064 | Train 0.9791 | Val 0.8169


Epoch 59/60: 100%|███████████████████████████████████████████████████████████████████| 140/140 [01:43<00:00,  1.36it/s]


Loss 10.594 | Train 0.9807 | Val 0.8169


Epoch 60/60: 100%|███████████████████████████████████████████████████████████████████| 140/140 [01:48<00:00,  1.29it/s]


Loss 10.062 | Train 0.9825 | Val 0.8151
Best Val Accuracy: 0.8168761220825853


In [2]:
# ==============================
# TEST EVALUATION
# ==============================

model.load_state_dict(torch.load("best_sign_transformer.pth", map_location=DEVICE))
model.eval()

all_preds = []
all_labels = []

correct, total = 0, 0

with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Testing"):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)

        preds = out.argmax(dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y.cpu().numpy())

        correct += (preds == y).sum().item()
        total += y.size(0)

test_acc = correct / total
print(f"\n🎯 Test Accuracy: {test_acc:.4f}")

# Detailed per-class metrics
print("\n📊 Classification Report:")
print(
    classification_report(
        all_labels,
        all_preds,
        target_names=le.classes_,
        digits=4
    )
)


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 18/18 [00:02<00:00,  6.65it/s]


🎯 Test Accuracy: 0.8133

📊 Classification Report:
              precision    recall  f1-score   support

       about     1.0000    0.8333    0.9091         6
       after     0.6000    1.0000    0.7500         3
       angry     1.0000    1.0000    1.0000         3
       apple     0.6000    1.0000    0.7500         3
        aunt     0.5000    0.6667    0.5714         3
        baby     0.8571    1.0000    0.9231         6
         bad     0.5000    0.6667    0.5714         3
    bathroom     0.6667    0.6667    0.6667         3
      before     1.0000    1.0000    1.0000         3
         big     0.7500    1.0000    0.8571         3
        bird     1.0000    1.0000    1.0000         3
        blue     0.7500    1.0000    0.8571         3
         boy     1.0000    1.0000    1.0000         3
     brother     1.0000    1.0000    1.0000         3
       brown     1.0000    1.0000    1.0000         3
       brush     1.0000    1.0000    1.0000         3
         bug     1.0000    1.0